In [ ]:
from model_pytorch import *
import os 
import pickle
from cfg import Config
from tqdm import tqdm
import torch.nn.functional as F
from torch.utils.data import DataLoader
import time

In [ ]:
config = Config()

In [ ]:
def check_data():
    if os.path.isfile(config.p_path):
        print ('Loading existing data from {} model'.format(config.mode))
        with open(config.p_path,'rb') as handle:
            tmp = pickle.load(handle)
            return tmp
    else:
        return None

In [ ]:
def return_sample(dummy) :
    rand_class = np.random.choice(class_dist.index,p=prob_dist)
    file = np.random.choice(df[df.label==rand_class].index)
    rate, wav = wavfile.read('clean/'+file)
    label = df.at[file,'label']
    rand_index = np.random.randint(0,wav.shape[0]-config.step)
    sample = wav[rand_index:rand_index+config.step]
    X_sample = mfcc(sample,rate,numcep=config.nfeat
                   ,nfilt=config.nfilt,nfft=config.nfft)


    return X_sample,classes.index(label)


In [ ]:
def build_rand_feat():
    tmp = check_data()
    if tmp:
        print ('data exist')
        return tmp.data[0], tmp.data[1]
    
    X = []
    y = []
    _min,_max = float('inf'),-float('inf')
    import multiprocessing
    pool = multiprocessing.Pool(4)
    total_samples = 200000
    results= list(tqdm(pool.imap_unordered(return_sample,range(total_samples)),total=total_samples))
    pool.close()
#     for _ in tqdm(range(10000)):
#         rand_class = np.random.choice(class_dist.index,p=prob_dist)
#         file = np.random.choice(df[df.label==rand_class].index)
#         rate, wav = wavfile.read('clean/'+file)
#         label = df.at[file,'label']
#         rand_index = np.random.randint(0,wav.shape[0]-config.step)
#         sample = wav[rand_index:rand_index+config.step]
#         X_sample = mfcc(sample,rate,numcep=config.nfeat
#                        ,nfilt=config.nfilt,nfft=config.nfft)
        
# #         X_sample= logfbank(sample,rate,nfilt=config.nfilt,nfft=config.nfft).T
        
#         _min = min(np.amin(X_sample),_min)
#         _max = max(np.amax(X_sample),_max)
#         X.append(X_sample)
#         y.append(classes.index(label))
    for result in results :
        _min = min(np.amin(result[0]),_min)
        _max = max(np.amin(result[0]),_max)
        X.append(result[0])
        y.append(result[1])


    config.min = _min
    config.max = _max
    X,y = np.array(X), np.array(y)
    X = (X - _min) / (_max-_min)
#     if config.mode == 'conv':
#         X = X.reshape(X.shape[0],X.shape[1],X.shape[2],1)
#     elif config.mode == 'time':
#         X= X.reshape(X.shape[0],X.shape[1],X.shape[2])
    config.data = (X,y)
    
    with open('./pickels/conv.p','wb') as handle:
        pickle.dump(config,handle,protocol=2)
        
    return X,y

In [ ]:

# model.add(Conv2D(16,(3,3),activation='relu',strides=(1,1),padding='same',input_shape=input_shape))
# model.add(Conv2D(32,(3,3),activation='relu',strides=(1,1),padding='same'))
# model.add(Conv2D(64,(3,3),activation='relu',strides=(1,1),padding='same'))
# model.add(Conv2D(128,(3,3),activation='relu',strides=(1,1),padding='same'))
# model.add(MaxPool2D((2,2)))
# model.add(Dropout(0.5))
# model.add(Flatten())
# model.add(Dense(128,activation='relu'))
# model.add(Dense(64,activation='relu'))
# model.add(Dense(10,activation='softmax'))


In [ ]:
class ConvNet(torch.nn.Module):

    def __init__(self, num_classes):
        super(ConvNet, self).__init__()
        
      
        # 28x28x1 => 28x28x8
        self.conv_1 = torch.nn.Conv2d(in_channels=1,
                                      out_channels=8,
                                      kernel_size=(3, 3),
                                      stride=(1, 1),
                                      padding=1) 
        # 28x28x8 => 14x14x8
        self.pool_1 = torch.nn.MaxPool2d(kernel_size=(2, 2),
                                         stride=(2, 2),
                                         padding=0) # (2(14-1) - 28 + 2) = 0                                       
        # 14x14x8 => 14x14x16
        self.conv_2 = torch.nn.Conv2d(in_channels=8,
                                      out_channels=16,
                                      kernel_size=(3, 3),
                                      stride=(1, 1),
                                      padding=1) # (1(14-1) - 14 + 3) / 2 = 1                 
        # 14x14x16 => 7x7x16                             
        self.pool_2 = torch.nn.MaxPool2d(kernel_size=(2, 2),
                                         stride=(2, 2),
                                         padding=0) # (2(7-1) - 14 + 2) = 0
        
        
        
        self.conv_1 = torch.nn.Conv2d(in_channels=1,
                                  out_channels=16,
                                  kernel_size=(3, 3),
                                  stride=(1, 1),
                                  padding=1) 
        self.conv_2 = torch.nn.Conv2d(in_channels=16,
                              out_channels=32,
                              kernel_size=(3, 3),
                              stride=(1, 1),
                              padding=1) 
        self.conv_3 = torch.nn.Conv2d(in_channels=32,
                              out_channels=64,
                              kernel_size=(3, 3),
                              stride=(1, 1),
                              padding=1) 
        self.conv_4 = torch.nn.Conv2d(in_channels=64,
                              out_channels=128,
                              kernel_size=(3, 3),
                              stride=(1, 1),
                              padding=1)
        
        
        self.max_pool = torch.nn.MaxPool2d(kernel_size=(2, 2))
        

        self.linear_1 = torch.nn.Linear(3072, 128)

        self.linear_2 = torch.nn.Linear(128, 64)

        self.linear_3 = torch.nn.Linear(64, 10)

        
        
    def forward(self, x):
        out = self.conv_1(x)
        out = F.relu(out)
        out = self.conv_2(out)
        out = F.relu(out)
        out = self.conv_3(out)
        out = F.relu(out)
        out = self.conv_4(out)
        out = F.relu(out)
        out = self.max_pool(out)
        out = out.flatten(start_dim=1, end_dim=-1)
        out = self.linear_1(out)
        out = F.relu(out)
        out = self.linear_2(out)
        out = F.relu(out)
        out = self.linear_3(out)
        logits = F.relu(out)
        
        
        probas = F.softmax(logits, dim=1)
        return logits, probas

In [ ]:
X,y = build_rand_feat()

In [ ]:
model = ConvNet(10).to('cuda:0')

In [ ]:
train_loader = DataLoader(dataset=list(zip(X.reshape(200000,1,9,13),y)), 
                          batch_size=1000, 
                          shuffle=True)

In [ ]:
def compute_accuracy(model, data_loader):
    correct_pred, num_examples = 0, 0
    for features, targets in data_loader:
        features = features.to(device).float()
        targets = targets.to(device)
        logits, probas = model(features)
        _, predicted_labels = torch.max(probas, 1)
        num_examples += targets.size(0)
        correct_pred += (predicted_labels == targets).sum()
    return correct_pred.float()/num_examples * 100
    

start_time = time.time()
learning_rate = 0.01
num_epochs = 30
device= 'cuda'
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
for epoch in range(num_epochs):
    model = model.train()
    for batch_idx, (features, targets) in enumerate(train_loader):
        
        features = features.to(device)
        targets = targets.to(device)

        ### FORWARD AND BACK PROP
        logits, probas = model(features.float().cuda())


        cost = F.cross_entropy(logits, targets.cuda())

        
        optimizer.zero_grad()
        
        cost.backward()
        
        ### UPDATE MODEL PARAMETERS
        optimizer.step()
        
        ### LOGGING
        if not batch_idx % 50:
            print ('Epoch: %03d/%03d | Batch %03d/%03d | Cost: %.4f' 
                   %(epoch+1, num_epochs, batch_idx, 
                     len(train_loader), cost))
    
    model = model.eval()
    print('Epoch: %03d/%03d training accuracy: %.2f%%' % (
          epoch+1, num_epochs, 
          compute_accuracy(model, train_loader)))

    print('Time elapsed: %.2f min' % ((time.time() - start_time)/60))
    
print('Total Training Time: %.2f min' % ((time.time() - start_time)/60))